In [1]:
import os
import pickle
import tensorflow as tf
from tensorflow.keras import Input, Model, layers, optimizers, losses, metrics, datasets
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 忽略低级别警告

root_URL = 'C:\\Users\\lenovo\\Desktop\\'

In [2]:
# 加载数据
pickle_file = 'svhn_dataset_112x56.pkl'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_images = save['train_images']
    train_labels = save['train_labels']
    train_bboxes = save['train_bboxes']
    test_images = save['test_images']
    test_labels = save['test_labels']
    test_bboxes = save['test_bboxes']
    del save  # hint to help gc free up memory
    print('Train Shape:', train_images.shape, train_labels.shape, train_bboxes.shape)
    print('Test Shape:', test_images.shape, test_labels.shape, test_bboxes.shape)

Train Shape: (33401, 56, 112) (33401, 6) (33401, 4)
Test Shape: (13068, 56, 112) (13068, 6) (13068, 4)


In [3]:
# 预处理函数
def preprocess(x):
    x = tf.expand_dims(x, axis=-1) # 在axis=3扩展一维
    return x

In [4]:
train_images = preprocess(train_images)
test_images = preprocess(test_images)
print('Train Shape:', train_images.shape, train_labels.shape, train_bboxes.shape)
print('Test Shape:', test_images.shape, test_labels.shape, test_bboxes.shape)

Train Shape: (33401, 56, 112, 1) (33401, 6) (33401, 4)
Test Shape: (13068, 56, 112, 1) (13068, 6) (13068, 4)


In [5]:
# VGG16
inputs = Input(shape = (56,112,1))

#layer_1
model = layers.Conv2D(64,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu',kernel_initializer='uniform')(inputs)
model = layers.Conv2D(64,(3,3),strides=(1,1),padding='same',data_format='channels_last',kernel_initializer='uniform',activation='relu')(model)
model = layers.MaxPooling2D((2,2))(model)

#layer_2
model = layers.Conv2D(128,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu',kernel_initializer='uniform')(model)
model = layers.Conv2D(128,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu',kernel_initializer='uniform')(model)
model = layers.MaxPooling2D((2,2))(model)

#layer_3
model = layers.Conv2D(256,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu')(model)
model = layers.Conv2D(256,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu')(model)
model = layers.Conv2D(256,(1,1),strides=(1,1),padding='same', data_format='channels_last', activation='relu')(model)
model = layers.MaxPooling2D((2,2))(model)

#layer_4
model = layers.Conv2D(512,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu')(model)
model = layers.Conv2D(512,(3,3),strides=(1,1),padding='same', data_format='channels_last', activation='relu')(model)
model = layers.Conv2D(512,(1,1),strides=(1,1),padding='same', data_format='channels_last', activation='relu')(model)
model = layers.MaxPooling2D((2,2))(model)

#layer_5
model = layers.Conv2D(512,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu')(model)
model = layers.Conv2D(512,(3,3),strides=(1,1),padding='same', data_format='channels_last', activation='relu')(model)
model = layers.Conv2D(512,(1,1),strides=(1,1),padding='same', data_format='channels_last', activation='relu')(model)
model = layers.MaxPooling2D((2,2))(model)

model = layers.Flatten()(model)  #拉平
model = layers.Dense(512,activation='relu')(model)

outputs = layers.Dense(4)(model)

In [5]:
# 创建网络模型并装配模型
inputs = Input(shape = (56,112,1))

model = layers.BatchNormalization()(inputs)
model = layers.Conv2D(32, (7,7), padding='same', activation='relu')(model)
model = layers.MaxPool2D(pool_size=2)(model)

model = layers.BatchNormalization()(model)
model = layers.Conv2D(64, (5,5), padding='same', activation='relu')(model)
model = layers.MaxPool2D(pool_size=2)(model)

model = layers.BatchNormalization()(model)
model = layers.Conv2D(128, (3,3), padding='same', activation='relu')(model)
model = layers.MaxPool2D(pool_size=2)(model)

model = layers.Dropout(0.5)(model)

model = layers.Flatten()(model)
# model = layers.Dense(1024, activation='relu')(model)
model = layers.Dense(512, activation='relu')(model)

outputs = layers.Dense(4)(model)# bbox

In [6]:
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='MSE',
              optimizer="rmsprop",
              metrics=['mae'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 56, 112, 1)]      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 56, 112, 1)        4         
_________________________________________________________________
conv2d (Conv2D)              (None, 56, 112, 32)       1600      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 56, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 56, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 56, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 28, 64)        0     

In [7]:
# 设置回调功能
filepath = 'my_model_bbox_500.h5' # 保存模型地址
saved_model = tf.keras.callbacks.ModelCheckpoint(filepath, verbose = 1) # 回调保存模型功能
tensorboard = tf.keras.callbacks.TensorBoard() # 回调可视化数据功能

In [8]:
# 执行训练与验证
history = model.fit(train_images, train_bboxes, 
                    batch_size = 16, epochs = 500,
                    validation_split = 0.2,
                    callbacks = [saved_model, tensorboard])

Train on 26720 samples, validate on 6681 samples
Epoch 1/500
26688/26720 [============================>.] - ETA: 0s - loss: 75.9017 - mae: 6.1440
Epoch 00001: saving model to my_model_bbox_500.h5
26720/26720 [==============================] - 58s 2ms/sample - loss: 75.8379 - mae: 6.1411 - val_loss: 31.6331 - val_mae: 3.9191
Epoch 2/500
26704/26720 [============================>.] - ETA: 0s - loss: 36.9329 - mae: 4.3925
Epoch 00002: saving model to my_model_bbox_500.h5
26720/26720 [==============================] - 55s 2ms/sample - loss: 36.9255 - mae: 4.3922 - val_loss: 37.2047 - val_mae: 4.5635
Epoch 3/500
26688/26720 [============================>.] - ETA: 0s - loss: 31.2608 - mae: 3.9984
Epoch 00003: saving model to my_model_bbox_500.h5
26720/26720 [==============================] - 56s 2ms/sample - loss: 31.2771 - mae: 3.9986 - val_loss: 29.3357 - val_mae: 3.7095
Epoch 4/500
26704/26720 [============================>.] - ETA: 0s - loss: 28.0119 - mae: 3.7416
Epoch 00004: saving mod

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



26688/26720 [============================>.] - ETA: 0s - loss: 6.4445 - mae: 1.9200
Epoch 00455: saving model to my_model_bbox_500.h5
26720/26720 [==============================] - 57s 2ms/sample - loss: 6.4479 - mae: 1.9203 - val_loss: 18.6428 - val_mae: 2.7967
Epoch 456/500
26688/26720 [============================>.] - ETA: 0s - loss: 6.3895 - mae: 1.9115
Epoch 00458: saving model to my_model_bbox_500.h5
26720/26720 [==============================] - 57s 2ms/sample - loss: 6.3902 - mae: 1.9117 - val_loss: 18.9534 - val_mae: 2.8273
Epoch 459/500
26688/26720 [============================>.] - ETA: 0s - loss: 6.3177 - mae: 1.9031
Epoch 00459: saving model to my_model_bbox_500.h5
26720/26720 [==============================] - 57s 2ms/sample - loss: 6.3166 - mae: 1.9030 - val_loss: 19.0160 - val_mae: 2.8514
Epoch 460/500
26704/26720 [============================>.] - ETA: 0s - loss: 6.2837 - mae: 1.8956
Epoch 00460: saving model to my_model_bbox_500.h5
26720/26720 [======================